## Second Assignment: Segmenting and Clustering Neighborhoods in Toronto

*For answer to second question of the assignment skip to In[10]

First step: Install packages and import libraries

In [1]:
!conda install -c conda-forge wikipedia --yes
import pandas as pd
import wikipedia as wp
import numpy as np

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
wikipedia                 1.4.0                    py35_0    conda-forge


Second step: Retieve table from html source and format to read fisrt row as header

In [2]:
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
# Transform first column into header
df.columns=['PostalCode', 'Borough', 'Neighborhood']
df.drop(df.index[0], inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Third step: drop rows with borough not assigned

In [3]:
df=df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Third step: Group rows with identical postal code. First, in order to verify hoy many rows we should have in the end I used the group by, count() method by PostalCode

In [4]:
check=df.groupby('PostalCode').count()
check.shape

(103, 2)

This shows us there is a total of 103 different postal codes in the data frame
Then we group by postal code, but now we use the join function in order to mix different neighborhoods separated by a comma

In [5]:
new_df=df.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))
new_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we check that the total amount of rows is the same as the prdeicted one of 103

In [6]:
new_df.shape

(103, 3)

We can see that it is the same number of rows, so even though the order of the rows has changed the result is still correct

Fourth step: Replace not assigned values for neighborhood with value from borough

In [7]:
final_df=new_df
final_df['Neighborhood'].where(final_df.Neighborhood!='Not assigned', final_df.Borough, inplace=True)
final_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


To finish up, we check that there are no rows containing a Not assigned neighborhoood, that could have been joined with other neighborhoods when grouping by postal code

In [8]:
final_df[final_df['Neighborhood'].str.contains("assigned")]

,PostalCode,Borough,Neighborhood


And we verify once more the shaoe of the final dataframe

In [9]:
final_df.shape

(103, 3)

Second question of the assignment

First step is to retrieve the latitude and longitude of each postal code and assigning it in new columns labeled 'Latitude' and 'Longitude' to the data frame. Since I couldn't retrieve the latitude and longitude through the geocoder library, I will use the csv file and name it geo_data

In [10]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.rename(columns={'Postal Code':'PostalCode', 'Latitude':'Latitude', 'Longitude':'Longitude'}, inplace=True)
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then I merged the data frames using the merge method, to assign the correct latitude and longitude to each postal code

In [11]:
merged_df=pd.merge(final_df, geo_data, how='left', on='PostalCode')
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Scarborough Village West, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848
